** 0.0 / IMPORTS & INIT**

In [1]:
import pandas as pd
import time
import os

**1.1 / CSV IMPORTING**

We import the 4 working databases from csv files in the subdirectory "Base" at the root of the current working directory through Pandas and assign them as Dataframes.                          
We also display progress information, since the process can be quite lenghty.

Caution : When all bases are imported at once, they are all loaded into RAM and they can take up to 20G. If your machine can't handle it, you should comment out what you don't need and work sequentially.

In [2]:
# Imports all bases and displays progress info
# Note : last base uses a comma as separator

start = time.perf_counter() # getting starting timestamp
print('Starting import...\n\n')
print('Importing D_avantage...')
D_avantage = pd.read_csv("Base/declaration_avantage_2020_02_19_04_00.csv", sep = ";")
print('D_avantage successfully imported. 3 more to go.')
print('Importing D_Convention...')
D_Convention = pd.read_csv("Base/declaration_convention_2020_02_19_04_00.csv", sep = ";")
print('D_Convention successfully imported. 2 more to go.')
print('Importing D_Remuneration...')
D_Remuneration = pd.read_csv("Base/declaration_remuneration_2020_02_19_04_00.csv", sep = ";")
print('D_Remuneration successfully imported. 1 more to go.')
print('Importing Entreprise...')
Entreprise = pd.read_csv("Base/entreprise_2020_02_19_04_00.csv",sep = ",")
print('Entreprise successfully imported.')
success = time.perf_counter() # getting ending timestamp
import_time = int(success - start) 
print('All csv successfully imported in %s seconds.'%(import_time))

Starting import...


Importing D_avantage...
D_avantage successfully imported. 3 more to go.
Importing D_Convention...
D_Convention successfully imported. 2 more to go.
Importing D_Remuneration...
D_Remuneration successfully imported. 1 more to go.
Importing Entreprise...
Entreprise successfully imported.
All csv successfully imported in 163 seconds.


**1.2 / DATAFRAMES CLEANING**

We remove all columns that contain duplicate info (even under a different form) or are unnecessary for the desired analysis.

In [50]:
columns_ = ['identifiant', 'denomination_sociale']
ids_entreprise = pd.DataFrame(columns= columns_)

for i in Entreprise['identifiant']:
    if i not in ids_entreprise['identifiant']:
        chaise = Entreprise.loc[Entreprise['identifiant'] == i].index[0]
        armoire = []
        buffet = []
        armoire.append(Entreprise.iloc[chaise]['identifiant'])
        armoire.append(Entreprise.iloc[chaise]['denomination_sociale'])
        buffet.append(armoire)
        



        ids_entreprise.append(pd.DataFrame(buffet, columns=['identifiant', 'denomination_sociale']))

         #print(ids_entreprise)

print(ids_entreprise)











        



       # ids_entreprise['denomination_sociale'].appen














Empty DataFrame
Columns: [identifiant, denomination_sociale]
Index: []
